In [1]:
#file taken out from google colab. apologies for any bugs

#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#for data manipulation
import pandas as pd
import numpy as np

#for visualization
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

#filtering warnings
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

#for some stats
from scipy import stats
from scipy.stats import norm, skew #for some statistics


In [3]:
#for display

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.2f}'.format

In [6]:
#products = pd.read_excel("drive/My Drive/Datasets/sholyst/products.xlsx")
products = pd.read_excel("C:/Users/Administrator/Desktop/18125760079_yash_kanvinde/products.xlsx")
products.head(5)

ProductId  \
0  E8AC41D107EA5DBB5DF561A684899B38   
1  04AE770377AABCCBCCDD5FBC93CC83E1   
2  21B0CF144B3CFFDDCCE4937C7731A7CE   
3  129CE0C7596D2D346E5CDC54E55A3A67   
4  A5E51E0F6C9929ECD16D875EC9A4AFA5   

                                               Title  \
0            Color Dose Pearls Bronze 59 Nail Polish   
1     Rimmel Space Dust Nail Polish - Luna Love #004   
2       Color Dose Pearls Copper Rust 58 Nail Polish   
3                Color Dose Matt Nude 25 Nail Polish   
4  SUGAR Tip Tac Toe Nail Lacquer - 032 Mary Popp...   

                                           Image URL Division Category  \
0  https://media6.ppl-media.com/tr:h-750,w-750,c-...   Makeup    Nails   
1  https://media6.ppl-media.com/tr:h-750,w-750,c-...   Makeup    Nails   
2  https://media6.ppl-media.com/tr:h-750,w-750,c-...   Makeup    Nails   
3  https://media6.ppl-media.com/tr:h-235,w-235,c-...      NaN    Nails   
4  https://media6.ppl-media.com/tr:h-750,w-750,c-...   Makeup    Nails   

  Sub category Article type  Offer price  Sale price       Color Gender  
0  Nail Polish  Nail Polish           59          59  Color Dose  women  
1  Nail Polish  Nail Polish          310         310      Rimmel  women  
2  Nail Polish  Nail Polish           59          59  Color Dose  women  
3  Nail Polish  Nail Polish           59          59  color dose  women  
4  Nail Polish  Nail Polish          237         249       Sugar  women

In [7]:
#just in case
#import surprise

ModuleNotFoundError: No module named 'surprise'

In [8]:
#lets not loose original data as  google colab takes time to load data.

products_copy = products.copy()

#lets start with getting the feel of data.
products_copy.shape

(7226, 11)

In [9]:
products_copy.nunique()

ProductId       7226
Title           6785
Image URL       7207
Division           9
Category          38
Sub category     100
Article type     100
Offer price      996
Sale price       752
Color            628
Gender             2
dtype: int64

In [10]:
products_copy.isnull().sum()/products_copy.shape[0]*100

ProductId       0.00
Title           0.00
Image URL       0.00
Division       33.13
Category        0.00
Sub category    0.00
Article type    0.00
Offer price     0.00
Sale price      0.00
Color           1.49
Gender          0.00
dtype: float64

In [11]:
#lets fill missing values for color as it is hardly 1%
#we will use KNN predicting them with better accuracy


#lets try filling days values

impute =  products_copy.drop(['Division','ProductId',	'Title','Image URL','Sub category',	'Article type'	],axis = 1)
encode_p     = impute['Category']
from sklearn import preprocessing
le           = preprocessing.LabelEncoder()
le_encode  = le.fit(encode_p)
le_encode  = le.transform(encode_p) 
le_encode

impute['num_category'] = le_encode
print(impute.shape)
impute = impute.drop('Category', axis = 1)
print(impute.shape)

train_x_impute    = impute[~pd.isnull(impute['Color'])].drop('Color', axis = 1)
test_x_impute    = impute[pd.isnull(impute['Color'])].drop('Color', axis = 1)
print(train_x_impute.shape)
print(test_x_impute.shape)


train_knn_dummy  = pd.get_dummies(train_x_impute)
test_knn_dummy   = pd.get_dummies(test_x_impute)
print(train_knn_dummy.shape, test_knn_dummy.shape)

#making train_y
train_y_knn       = impute.loc[train_x_impute.index,'Color']


from sklearn.preprocessing import StandardScaler
scaler_impute          = StandardScaler()
scaler_impute.fit(train_knn_dummy)
train_x_impute_scaled  = scaler_impute.transform(train_knn_dummy)
test_x_impute_scaled   = scaler_impute.transform(test_knn_dummy)

from sklearn.neighbors import KNeighborsClassifier
model_knn              = KNeighborsClassifier(n_neighbors=7)
model_knn.fit(train_x_impute_scaled, train_y_knn)
model_knn              = model_knn.predict(test_x_impute_scaled)

#putting into original data
products_copy.loc[test_x_impute.index, 'Color'] = model_knn







(7226, 6)
(7226, 5)
(7118, 4)
(108, 4)
(7118, 5) (108, 5)


In [12]:
products_copy.isnull().sum()

ProductId          0
Title              0
Image URL          0
Division        2394
Category           0
Sub category       0
Article type       0
Offer price        0
Sale price         0
Color              0
Gender             0
dtype: int64

In [13]:
#lets try to build a basic model which will give output on basis of division

def build_chart(division):
    filt_result = products_copy[products_copy['Division'] == division]
    
    return filt_result

In [14]:
build_chart('Makeup').head(5)

ProductId  \
0  E8AC41D107EA5DBB5DF561A684899B38   
1  04AE770377AABCCBCCDD5FBC93CC83E1   
2  21B0CF144B3CFFDDCCE4937C7731A7CE   
4  A5E51E0F6C9929ECD16D875EC9A4AFA5   
5                   C1E30881C95BF8C   

                                               Title  \
0            Color Dose Pearls Bronze 59 Nail Polish   
1     Rimmel Space Dust Nail Polish - Luna Love #004   
2       Color Dose Pearls Copper Rust 58 Nail Polish   
4  SUGAR Tip Tac Toe Nail Lacquer - 032 Mary Popp...   
5               LYN Nail Polish Ginger Spice (12 ml)   

                                           Image URL Division Category  \
0  https://media6.ppl-media.com/tr:h-750,w-750,c-...   Makeup    Nails   
1  https://media6.ppl-media.com/tr:h-750,w-750,c-...   Makeup    Nails   
2  https://media6.ppl-media.com/tr:h-750,w-750,c-...   Makeup    Nails   
4  https://media6.ppl-media.com/tr:h-750,w-750,c-...   Makeup    Nails   
5  https://media6.ppl-media.com/tr:h-750,w-750,c-...   Makeup    Nails   

  Sub category Article type  Offer price  Sale price       Color Gender  
0  Nail Polish  Nail Polish           59          59  Color Dose  women  
1  Nail Polish  Nail Polish          310         310      Rimmel  women  
2  Nail Polish  Nail Polish           59          59  Color Dose  women  
4  Nail Polish  Nail Polish          237         249       Sugar  women  
5  Nail Polish  Nail Polish          350         350         LYN  women

In [15]:
#we need to fix division for further analysis. We could divide data in train,test,validate to increase the overall accuracy but due to time constaints we will do it only once 
#and assume that we have fixed it correctly


#lets try filling division values

impute =  products_copy.drop(['ProductId',	'Title','Image URL','Sub category',	'Article type'	],axis = 1)
encode_p     = impute['Category']
from sklearn import preprocessing
le           = preprocessing.LabelEncoder()
le_encode  = le.fit(encode_p)
le_encode  = le.transform(encode_p) 
le_encode

impute['num_category'] = le_encode

encode_c     = impute['Color']
from sklearn import preprocessing
le           = preprocessing.LabelEncoder()
le_encode  = le.fit(encode_c)
le_encode  = le.transform(encode_c) 
le_encode

impute['num_color'] = le_encode



print(impute.shape)
impute = impute.drop(['Category','Color'], axis = 1)
print(impute.shape)

train_x_impute    = impute[~pd.isnull(impute['Division'])].drop('Division', axis = 1)
test_x_impute    = impute[pd.isnull(impute['Division'])].drop('Division', axis = 1)
print(train_x_impute.shape)
print(test_x_impute.shape)


train_knn_dummy  = pd.get_dummies(train_x_impute)
test_knn_dummy   = pd.get_dummies(test_x_impute)
print(train_knn_dummy.shape, test_knn_dummy.shape)

#making train_y
train_y_knn       = impute.loc[train_x_impute.index,'Division']


from sklearn.preprocessing import StandardScaler
scaler_impute          = StandardScaler()
scaler_impute.fit(train_knn_dummy)
train_x_impute_scaled  = scaler_impute.transform(train_knn_dummy)
test_x_impute_scaled   = scaler_impute.transform(test_knn_dummy)

from sklearn.neighbors import KNeighborsClassifier
model_knn              = KNeighborsClassifier(n_neighbors=7)
model_knn.fit(train_x_impute_scaled, train_y_knn)
model_knn              = model_knn.predict(test_x_impute_scaled)

#putting into original data
products_copy.loc[test_x_impute.index, 'Division'] = model_knn







(7226, 8)
(7226, 6)
(4832, 5)
(2394, 5)
(4832, 6) (2394, 6)


In [16]:
products_copy.isnull().sum()

ProductId       0
Title           0
Image URL       0
Division        0
Category        0
Sub category    0
Article type    0
Offer price     0
Sale price      0
Color           0
Gender          0
dtype: int64

In [17]:
#lets try building a simple solution telling us unique items

def sim(i):
    a=products_copy[i].value_counts()
    a=pd.DataFrame(a)
    b=a.index
    for i in b:
        a=products_copy[products_copy.Division==i][['Title']].Title.unique()
        a=pd.DataFrame(a)
        print(i,a.head(10))
        print('---------------------------------------------')        

        

In [18]:
#we get simple results
sim('Division')


Body                                                    0
0  Eylina Moisture Matte Lipstick Radiant Orchid ...
1  Blue Heaven Combo Of 2 Xpression Lipstick (R 0...
2  Barva Skin Therapie Lipstick - 504 (Paraben Fr...
3  Blue Heaven Combo Of 2 Innocence Lipsticks (Or...
4  Essence Xxxl Nude Lipgloss 03 Taste The Sweets...
5  Eylina Sheer Shine Lip Gloss Jellyfish Pink LI...
6  Eylina Sheer Shine Lip Gloss Tangerine LIG006 ...
7   FACES Go Chic Lip Gloss Rose Shimmer 15 (7.5 ml)
8    Essence Xxxl Shine Lipgloss 01 Pure chic (5 ml)
9  Eylina Sheer Shine Lip Gloss Orange LIG002 (6....
---------------------------------------------
Makeup                                                    0
0            Color Dose Pearls Bronze 59 Nail Polish
1     Rimmel Space Dust Nail Polish - Luna Love #004
2       Color Dose Pearls Copper Rust 58 Nail Polish
3                Color Dose Matt Nude 25 Nail Polish
4  SUGAR Tip Tac Toe Nail Lacquer - 032 Mary Popp...
5               LYN Nail Polish Ginger Sp

In [19]:
#The code runs but takes too long. so for the time being i am taking 150 rows. please remove this line and run the code

products_copy = products_copy.head(150)

In [24]:
# there is not 

def shoplyst(product_id):
    #making dynamic product ID
    save = products_copy[products_copy['ProductId'] == product_id]['Division']
    
    save = save.tolist()
    
    #filtering via that ID 
    
    df   = products_copy[products_copy['Division'] == save[0]]
    #df_2 = df.drop([])
    print("subset df {}".format(df.shape))
    
    #using tfidf for Title column
    
    from sklearn.feature_extraction.text import TfidfVectorizer
    set_tfidf = df['Title'] 
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')

    tfidd_fit   = tfidf_vectorizer.fit(set_tfidf)
    tfidf       = tfidf_vectorizer.transform(set_tfidf)
    df_tfidf    = pd.SparseDataFrame(tfidf)
    print("tfidf shape {}".format(df_tfidf.shape))
    
    #encoding all categorial variable for that subset data
    
    
    encode_cat   = df['Category']
    from sklearn import preprocessing
    le           = preprocessing.LabelEncoder()
    le_encode    = le.fit(encode_cat)
    le_encode    = le.transform(encode_cat) 
    le_encode
    df['num_category'] = le_encode
    
    
    encode_su_c  = df['Sub category']
    le           = preprocessing.LabelEncoder()
    le_encode    = le.fit(encode_su_c)
    le_encode    = le.transform(encode_su_c)
    df['num_sub_cat'] = le_encode     
    
    encode_c     = df['Color']
    le           = preprocessing.LabelEncoder()
    le_encode    = le.fit(encode_c)
    le_encode    = le.transform(encode_c)
    df['num_color'] = le_encode
    
    encode_div   = df['Division']
    le           = preprocessing.LabelEncoder()
    le_encode    = le.fit(encode_div)
    le_encode    = le.transform(encode_div)
    
    df['num_div'] = le_encode
    
    df['num_gen'] = np.where(df['Gender'] == 'women', 1,0)
    
    
    df_2 = df.drop(['ProductId'	,'Title',	'Image URL','Division'	,'Category'	,'Sub category'	,'Article type','Gender','Color'], axis = 1)
    
    
    
    df_2 = df_2.reset_index()
    df_tfidf = df_tfidf.reset_index()
    
    final_df = pd.concat([df_2,df_tfidf ], axis = 1)
    
    
    
    print("final concat df shape {}".format(final_df.shape))
    
    import scipy
    
    sparse_df = scipy.sparse.csr_matrix(final_df.values)
    print(sparse_df.shape)
    
    
    #I frankly dont know why i am facing missing values. as i imputed all values. i have never worked on tfidf and clustering together
    
    
    from sklearn.preprocessing import Imputer
    data_imputer = Imputer(missing_values='NaN', strategy='mean', axis=0)
    # fit the imputer - suppose missing data is in the 0th column
    data_imputer = data_imputer.fit(sparse_df)
    # transform the data
    sparse_df = data_imputer.transform(sparse_df)

    
    #creating clusters to suggest later
    
    from sklearn.cluster import KMeans
    kmeanModel        = KMeans(n_clusters=3)
    kmeanModel.fit(sparse_df)
    kmeanModel.fit(sparse_df)
    lab               = kmeanModel.labels_
    
    df['lab']   = lab
    print("df_with_clust{}" .format(df.shape))
    
    check_clust = df[df['ProductId'] == product_id]['lab']
    print("lab{}".format(check_clust.shape))
    
    
    check_clust = check_clust.tolist()
    
    
     
    
    final_df    = df[df['lab'] == check_clust[0]]
    print("final products{}".format(final_df['Title'].head(10))) #display 10 similar products
    #print(final_df['Title'])
    
    

    
    
    
    
    

    
    

    
    
    
    

In [25]:
shoplyst('E8AC41D107EA5DBB5DF561A684899B38')

subset df (124, 11)
tfidf shape (124, 406)
final concat df shape (124, 415)
(124, 415)


C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


df_with_clust(124, 17)
lab(1,)
final products0              Color Dose Pearls Bronze 59 Nail Polish
1       Rimmel Space Dust Nail Polish - Luna Love #004
2         Color Dose Pearls Copper Rust 58 Nail Polish
3                  Color Dose Matt Nude 25 Nail Polish
4    SUGAR Tip Tac Toe Nail Lacquer - 032 Mary Popp...
5                 LYN Nail Polish Ginger Spice (12 ml)
6           LYN Nail Polish Sunset By The Lake (12 ml)
7    Stay Quirky Nail Polish, Mauve Monologue 99 (6...
8    Stay Quirky Nail Polish, Maroon - Not For Scho...
9    Sally Hansen Diamond Strength No Chip Nc-Black...
Name: Title, dtype: object


In [30]:
products_copy[products_copy['ProductId'] ==  'E8AC41D107EA5DBB5DF561A684899B38']

ProductId                                    Title  \
0  E8AC41D107EA5DBB5DF561A684899B38  Color Dose Pearls Bronze 59 Nail Polish   

                                           Image URL Division Category  \
0  https://media6.ppl-media.com/tr:h-750,w-750,c-...   Makeup    Nails   

  Sub category Article type  Offer price  Sale price       Color Gender  
0  Nail Polish  Nail Polish           59          59  Color Dose  women

In [ ]:
#THANK YOU!!!!

In [46]:
products_copy['new'] = pd.cut(products_copy['Sale price'],bins=350, labels=False)
products_copy[['Sale price','new']].head(100)

Sale price  new
0           59    0
1          310   29
2           59    0
3           59    0
4          249   22
5          350   34
6          350   34
7           88    3
8          115    6
9          395   39
10         158   11
11         400   40
12          88    3
13         575   61
14         499   52
15        1680  192
16         179   14
17          88    3
18        2999  349
19         549   58
20         390   39
21         115    6
22         229   20
23         229   20
24         680   73
25         109    5
26          88    3
27         158   11
28         430   44
29         399   40
30         208   17
31         275   25
32         199   16
33         300   28
34         249   22
35         229   20
36         400   40
37          62    0
38         249   22
39         149   10
40         390   39
41         300   28
42         300   28
43        1365  155
44         135    9
45         190   15
46         199   16
47         275   25
48         249   22
49         650   70
50         158   11
51         680   73
52          69    1
53         275   25
54         390   39
55         275   25
56         310   29
57         198   16
58         400   40
59        2999  349
60         275   25
61         549   58
62          88    3
63         460   47
64         135    9
65         179   14
66         400   40
67         199   16
68         199   16
69         430   44
70         390   39
71         158   11
72          88    3
73         150   10
74         229   20
75         275   25
76         350   34
77         350   34
78         350   34
79          88    3
80         199   16
81         400   40
82         172   13
83         395   39
84          99    4
85         240   21
86          62    0
87         300   28
88          99    4
89         199   16
90         549   58
91         135    9
92         275   25
93        1360  154
94         135    9
95         395   39
96          99    4
97        1360  154
98         499   52
99         350   34

In [47]:
products_copy['new'].nunique()

49

In [37]:
products_copy[products_copy['Sale price'] == 158]


ProductId  \
10  04DFF37DB3545CD0F6E28B31A5D263F6   
27  F836E69FD59CEDA67476E72462C90583   
50  70EDC4391BE120C7298C4FDB1C21450F   
71  466517B7903A8325C415B2F980C22C77   

                                                Title  \
10  Stay Quirky Nail Polish, Gel Finish, Purple Ge...   
27  Stay Quirky Nail Polish, Gel Finish, The Girl ...   
50  Stay Quirky Nail Polish, Gel Finish, Maroon Go...   
71  Stay Quirky Nail Polish, Gel Finish, Purple Ex...   

                                            Image URL Division Category  \
10  https://media6.ppl-media.com/tr:h-750,w-750,c-...   Makeup    Nails   
27  https://media6.ppl-media.com/tr:h-750,w-750,c-...   Makeup    Nails   
50  https://media6.ppl-media.com/tr:h-750,w-750,c-...   Makeup    Nails   
71  https://media6.ppl-media.com/tr:h-750,w-750,c-...   Makeup    Nails   

   Sub category Article type  Offer price  Sale price        Color Gender  \
10  Nail Polish  Nail Polish           99         158  Stay Quirky  women   
27  Nail Polish  Nail Polish           99         158  Stay Quirky  women   
50  Nail Polish  Nail Polish          100         158  Stay Quirky  women   
71  Nail Polish  Nail Polish           79         158  Stay Quirky  women   

     new  
10 17.00  
27 17.00  
50 17.00  
71 17.00